In [6]:
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, select, and_
from sqlalchemy.orm import sessionmaker
from scipy import stats
from numpy import mean
Base = declarative_base()

In [7]:
engine = create_engine('sqlite:///Resources/project3.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
class SqFeetData(Base):
    __tablename__ = 'sqfeet_data'
    state = Column(String, primary_key=True)
    median_home_price = Column(Integer)  
    median_home_size = Column(Integer)   
    income_data_state = Column(String, ForeignKey('income_data.state'))
    cities_data_state = Column(String, ForeignKey('cities_data.state'))
    income_data = relationship("IncomeData", back_populates="sqfeet_data", foreign_keys=[income_data_state])
    cities_data = relationship("CitiesData", back_populates="sqfeet_data", foreign_keys=[cities_data_state])

class IncomeData(Base):
    __tablename__ = 'income_data'
    state = Column(String, primary_key=True)
    earners_1 = Column(Integer)
    earners_2 = Column(Integer)
    earners_3 = Column(Integer)
    earners_4 = Column(Integer)
    sqfeet_data_state = Column(String, ForeignKey('sqfeet_data.state'))
    sqfeet_data = relationship("SqFeetData", back_populates="income_data")

class CitiesData(Base):
    __tablename__ = 'cities_data'
    state = Column(String, primary_key=True)
    region = Column(String)
    acronyms = Column(String)
    price = Column(Integer)
    sqfeet_data_state = Column(String, ForeignKey('sqfeet_data.state'))
    sqfeet_data = relationship("SqFeetData", back_populates="cities_data")


In [9]:
sqfeet_data_state = Column(String, ForeignKey('sqfeet_data.state'))
cities_data_state = Column(String, ForeignKey('cities_data.state'))

sqfeet_data = relationship("SqFeetData", back_populates="income_data", foreign_keys=[sqfeet_data_state])
cities_data = relationship("CitiesData", back_populates="income_data", foreign_keys=[cities_data_state])